In [13]:
from datetime import datetime
print(f'Päivitetty {datetime.now().strftime("%d.%m.%Y")} / Tatu Erkinjuntti')

Päivitetty 17.10.2024 / Tatu Erkinjuntti


Oppillaan kommentti: Itse en pidä yhdysvaltaistyylisestä päiväyksen formatoinnista kun ohjelma suoritetaan/luetaan tilanteessa jossa lukija on tottunut "perinteiseen" formatointiin => "day"."month"."year", kuten Suomessa yleisesti käytössä. Tästä syystä datetime formatointi on päivitetty tehtävässäni.

# Tehtävä 1.

## Kuvaus

Tehtävässä 1 on tarkoituksena soveltaa kuvailevan ja selittävän analytiikan menetelmiä valitsemiisi datoihin. Voit käyttää alempana mainittuja data-paketin datoja tai omaan työhön tai kiinnnostuksen kohteisiin liittyviä datoja. Käytä tehtävässä vähintään kahta eri datasettiä ja huolehdi että saat mukaan sekä kategorisia että määrällisiä muuttujia.

## Tehtävän kriteerit

Täysiin pisteisiin (5) pääset soveltamalla kaikkia esimerkkimuistioissa esitettyjä menetelmiä:

 - frekvenssitaulukot,
 - luokitellut jakaumat,
 - tilastolliset tunnusluvut, ristiintaulukoinnit,
 - tunnuslukujen vertailut ja korrelaatiot
 - tulosten visualisointeja.




## Tehtävän alustus

Haetaan tarvittavat kirjastos ja määritellään käytetyt data-aineistot. Itse teen tämän aina kerralla ohjelman alussa, näin yhdellä silmäyksellä näkee ohjelman riippuvaisuudet. Jos käytössä olisi pysyviä muuttujia, tulisi ne myös alustaa samalla.  

In [14]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

from scipy.stats import pearsonr
from scipy.stats import ttest_ind

sns.set_style('dark')

data_titanic = 'Data/titanic_new.xlsx'
data_kunnat = 'Data/kunnat.xlsx'
data_sales = "sales_data_sample.csv"